<a href="https://colab.research.google.com/github/bigboyfreezy/recommender-machinelearning/blob/main/CollaborativeRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Collaborative
#these systems are widely used, and they try to predict the rating or preference that a user would give an
#item-based on past ratings and preferences of other users. Collaborative filters do not require item metadata like its content-based counterparts.
import pandas
user_details = pandas.read_csv("https://modcom.co.ke/datasets/file.csv")
user_details.head(1)

,user_id,item_id,rating,timestamp
0,0,50,5,881250949


In [ ]:
movie_details = pandas.read_csv("https://modcom.co.ke/datasets/Movie_Id_Titles.csv")
movie_details.head(1)

,item_id,title
0,1,Toy Story (1995)


In [ ]:
#Merge both the movies and users with item id
data = pandas.merge(user_details,movie_details,on='item_id')
data.head(2)

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)


In [ ]:
#Find the mean of rating of each movie in descending order
data.groupby('title')['rating'].mean().sort_values(ascending=False).head(2)

title
They Made Me a Criminal (1939)                5.0
Marlene Dietrich: Shadow and Light (1996)     5.0
Name: rating, dtype: float64

In [ ]:
#Find how many times each movie was rated in descending order
data.groupby('title')['rating'].count().sort_values(ascending=False).head(2)

title
Star Wars (1977)    584
Contact (1997)      509
Name: rating, dtype: int64

In [ ]:
mean_rating= pandas.DataFrame(data.groupby('title')['rating'].mean())
mean_rating['num_of_ratings']=pandas.DataFrame(data.groupby('title')['rating'].count())
mean_rating.head(2)

,rating,num_of_ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5


In [ ]:
#Try to discover which movies did each user rate and what rating did he or she give
pivot = data.pivot_table(index='user_id',columns='title',values='rating')
pivot.head(2)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN


In [ ]:
#Lets correlate with the pivot table
select_movie=pivot['Godfather, The (1972)']

#Select similar movies based with the selected movies
similar=pivot.corrwith(select_movie)
similar_df=pandas.DataFrame(similar,columns=['Correlation'])
similar_df.sort_values('Correlation',ascending=False).head(2)


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Correlation
title,
Dark City (1998),1.0
8 Seconds (1994),1.0


In [ ]:
#Join the correlations with the no.of.ratings
similar_df = similar_df.join(mean_rating['num_of_ratings'])
similar_df.head(1)

,Correlation,num_of_ratings
title,,
'Til There Was You (1997),0.612372,9


In [ ]:
similar_df[similar_df['num_of_ratings'] > 200].sort_values('Correlation', ascending = False).head(20)


,Correlation,num_of_ratings
title,,
"Godfather, The (1972)",1.000000,413
"Godfather: Part II, The (1974)",0.683862,209
GoodFellas (1990),0.421477,226
"People vs. Larry Flynt, The (1996)",0.393439,215
Apocalypse Now (1979),0.374378,221
Dead Man Walking (1995),0.360525,299
Psycho (1960),0.336903,239
Field of Dreams (1989),0.309903,212
2001: A Space Odyssey (1968),0.305717,259
